#  Load libraries and options


In [ ]:
library(data.table)
library(glue)
library(dplyr)
library(plotly)
library(ggplot2)
library(grid)
library(gridExtra)
library(rvest)
library(ggrepel)
library(HGNChelper)
library(GenomicRanges)
library(splitstackshape)
library(parallel)
library(repr)
options(scipen=999)

# Load and prepare data

## OMIM Data

In [ ]:
# Load data from OMIM genemap2 file from 26-04-2021
genemap2 <- fread("/data/POL_data/genemap2.txt", skip="# Chromosome")
# Get phenotypes from genemap2
phenotypes <- strsplit(genemap2$Phenotypes, ";")
pheno2 <- lapply(1:length(phenotypes), function(i){
    xx <- phenotypes[[i]]
    xx2 <- rbindlist(lapply(xx, function(x){        
        y <- strsplit(x, ",")[[1]]
        inheritance <- y[length(y)]
        if(length(y)<2){
            omimNr <- "-"
            } else {
            omimNr <- y[length(y)-1]
            }
        disease <- y[1]
        row = genemap2[i,]
        row$disease=disease
        row$inheritance=inheritance
        row$omimNr=omimNr
        row
    }))
    xx2
})
pheno3 <- rbindlist(pheno2)
# Extract only AR genes from list
idx <- (grep("Autosomal recessive", pheno3$inheritance))
pheno4 <- pheno3[idx,]
# Only keep entries with known molecular defect based on OMIM Phenotype Mapping Method
types <- sapply(strsplit(pheno4$omimNr, " "), function(x){x[3]})
pheno5 <- pheno4[which( types == "(3)" | types == "(1)"),]
# Save generated data
fwrite(pheno5[,c(1,2,3,9),with = FALSE], "/data/POL_data/recessive_genes.bed", col.names=FALSE, row.names=FALSE, sep="\t")

## POL cohort data

In [ ]:
# Read input VCF
tgp_vcf <- fread("/data/POL_data/multisample_20210716.dv.bcfnorm.filt.unrelated.nogt.vcf.gz", skip="#CHROM", sep="\t")
# Read TGP VEP annotated file
tgp_vep <- fread("/data/POL_data/TGP-PL_subset_sorted_VEP.tsv", sep="\t")
# Split INFO from vcf file
tgp_vcf[, c("AF", "AQ", "AN", "AC") := tstrsplit(INFO, ";", fixed=TRUE)]
# Clean new columns, remove AQ (dupli
cates QUAL)
tgp_vcf[, AF := gsub("[^0-9.-]", "", AF)][, AN := gsub("[^0-9.-]", "", AN)][, AC := gsub("[^0-9.-]", "", AC)][, AQ := NULL]
# Create key - further used to merge with VCF file
tgp_vep[, key2 := gsub(">", "_", gsub(":", "_", key))]
setkey(tgp_vep, key2)
setkey(tgp_vcf, ID)
# Copy selected columns from VCF to TSV
selected_columns <- c("REF", "ALT", "QUAL", "AF", "AN", "AC")
tgp_vep[tgp_vcf, (selected_columns) := mget(paste0('i.', selected_columns))]
# Read AR genes from OMIM
omim_recessive <- fread("/data/POL_data/recessive_genes.bed", sep="\t", col.names=c("chr", "start", "end", "symbol"))
omim_recessive <- unique(omim_recessive, by = "symbol")
# Select variant from recessive genes only
tgp_recessive <- tgp_vep[which((Symbol %in% omim_recessive$symbol) & AC!=0)]
# Select variants with functional impact
tgp_vep_all <- tgp_recessive[which(Impact %in% c("MODERATE", "HIGH", "MODIFIER"))]
# Save generated data
fwrite(tgp_vep_all, "/data/POL_data/tgp_vep_all_subset_2021_07_16.tsv", sep="\t")
# Get only P/LP variants from ClinVar
tgp_clinvar <- tgp_vep_all[which(CLNSIG %in% c("Pathogenic", "Pathogenic/Likely_pathogenic", "Likely_pathogenic"))]
idx_duplicated_tgp <- which(duplicated(tgp_clinvar$key))
tgp_clinvar <- tgp_clinvar[-idx_duplicated_tgp,]
# Save genrated data
fwrite(tgp_clinvar, "/data/POL_data/tgp_clinvar_all_subset.tsv", sep="\t")

## gnomAD v3.0 annotated data

In [ ]:
# Read gnomAD v3.0 VEP annotated subset file (only variants from recessive genes)
gnomad_vep <- fread("/data/POL_data/gnomad_genome_v3.0_gt_subset_regulatory_sorted_VEP.tsv", sep="\t")
# Select variants with functional impact
gnomad_vep_all <- gnomad_vep[which(Impact %in% c("MODERATE", "HIGH", "MODIFIER"))]
# Remove dupliacted varinats
idx_duplicated <- which(duplicated(gnomad_vep_all$key))
gnomad_vep_all <- gnomad_vep_all[-idx_duplicated,]
# Save generated data
fwrite(gnomad_vep_all, "/data/POL_data/gnomad_vep_all_2021_07_16_1.tsv", sep="\t")
# Get only P/LP variants from ClinVar
gnomad_clinvar <- gnomad_vep_all[which(CLNSIG %in% c("Pathogenic", "Pathogenic/Likely_pathogenic", "Likely_pathogenic"))]
idx_duplicated_gnomad <- which(duplicated(gnomad_clinvar$key))
gnomad_clinvar <- gnomad_clinvar[-idx_duplicated_gnomad,]
# Select only variants present in NFE population
gnomad_clinvar <- gnomad_clinvar[which(gnomAD_AF_NFE != 0),]
# Save generated data
fwrite(gnomad_clinvar, "/data/POL_data/gnomad_clinvar_all.tsv", sep="\t")

## PanelApp data

In [ ]:
# Get data from PanelApp website
panels <- read_html("https://panelapp.genomicsengland.co.uk/panels/")
panels2 <- panels %>% html_nodes("h4")
# Extract genes from all panels
allPanelsList <- (lapply(panels2[-length(panels2)], function(panel){
    panelURL <- paste0("https://panelapp.genomicsengland.co.uk", panel %>% html_nodes("a") %>%  html_attr("href"), "download/01234/")
    print(panelURL)
    fread(panelURL)
    }))
allPanelsDT <- rbindlist(allPanelsList)
# Select genes reviewed and marked as "Green" for given panel
idx <- grep("Expert Review Green", allPanelsDT$"Sources(; separated)")
allPanelsDT2 <- allPanelsDT[idx,]
# Select genes with AR mode of inheritance
allPanelsDT3 <- allPanelsDT2[grep("BIALLELIC", allPanelsDT2$Model_Of_Inheritance),]
# Update gene symbols
updatedGenePanelSymbols <- checkGeneSymbols(allPanelsDT3$"Gene Symbol")
idx_duplicated_panels <- which(duplicated(updatedGenePanelSymbols$x))
updatedGenePanelSymbols <- updatedGenePanelSymbols[-idx_duplicated_panels,]
merge_allPanelsDT3 <- merge(allPanelsDT3, updatedGenePanelSymbols, by.x = "Gene Symbol", by.y = "x")
# Fix generated alias gene symbols
merge_allPanelsDT3[Suggested.Symbol == "EPRS1 /// QARS1"]$"Gene Symbol" <- "QARS1"
merge_allPanelsDT3[Suggested.Symbol == "CBLIF /// MIF /// MT3"]$"Gene Symbol" <- "CBLIF"
merge_allPanelsDT3$"Gene Symbol" <- ifelse(merge_allPanelsDT3$Approved == "FALSE", merge_allPanelsDT3$Suggested.Symbol, merge_allPanelsDT3$"Gene Symbol")
# Save generated data
fwrite(merge_allPanelsDT3, "/data/POL_data/allPanelsDT3_1.tsv", sep="\t")
allPanelsDT3 <- merge_allPanelsDT3

In [ ]:
# Select genes with AR mode of inheritance
allPanelsDT3 <- allPanelsDT2[grep("BIALLELIC", allPanelsDT2$Model_Of_Inheritance),]
# Update gene symbols
updatedGenePanelSymbols <- checkGeneSymbols(allPanelsDT3$"Gene Symbol")
idx_duplicated_panels <- which(duplicated(updatedGenePanelSymbols$x))
updatedGenePanelSymbols <- updatedGenePanelSymbols[-idx_duplicated_panels,]
merge_allPanelsDT3 <- merge(allPanelsDT3, updatedGenePanelSymbols, by.x = "Gene Symbol", by.y = "x")
# Fix generated alias gene symbols
merge_allPanelsDT3[Suggested.Symbol == "EPRS1 /// QARS1"]$"Gene Symbol" <- "QARS1"
merge_allPanelsDT3[Suggested.Symbol == "CBLIF /// MIF /// MT3"]$"Gene Symbol" <- "CBLIF"
merge_allPanelsDT3$"Gene Symbol" <- ifelse(merge_allPanelsDT3$Approved == "FALSE", merge_allPanelsDT3$Suggested.Symbol, merge_allPanelsDT3$"Gene Symbol")


## Fix gene symbols

In [ ]:
# Update P/LP ClinVar variants annotated by VEP with different gene symbol 
tgp_clinvar$clinvar_id <- gsub("/", "", gsub("https://www.ncbi.nlm.nih.gov/clinvar/variation/", "", tgp_clinvar$ClinVar_url))
gnomad_clinvar$clinvar_id <- gsub("/", "", gsub("https://www.ncbi.nlm.nih.gov/clinvar/variation/", "", gnomad_clinvar$ClinVar_url))
setkey(tgp_clinvar, clinvar_id)
setkey(gnomad_clinvar, clinvar_id)
clinvar_vcf_split_subset$ID <- as.character(clinvar_vcf_split_subset$ID)
setkey(clinvar_vcf_split_subset, ID)
# Merge data sets 
tgp_clinvar_updated_symbol <- tgp_clinvar[clinvar_vcf_split_subset]
gnomad_clinvar_updated_symbol <- gnomad_clinvar[clinvar_vcf_split_subset]
# Remove merged rows from ClinVar vcf
tgp_clinvar_updated_symbol <- na.omit(tgp_clinvar_updated_symbol, cols="Identifier")
gnomad_clinvar_updated_symbol <- na.omit(gnomad_clinvar_updated_symbol, cols="Identifier")
# Replace gene symbols if they differ
tgp_clinvar_updated_symbol$Symbol_001 <- ifelse(tgp_clinvar_updated_symbol$Symbol_001 %like% "ALLELEID", tgp_clinvar_updated_symbol$Symbol, tgp_clinvar_updated_symbol$Symbol_001)
gnomad_clinvar_updated_symbol$Symbol_001 <- ifelse(gnomad_clinvar_updated_symbol$Symbol_001 %like% "ALLELEID", gnomad_clinvar_updated_symbol$Symbol, gnomad_clinvar_updated_symbol$Symbol_001)
# Remove old gene symbol cols and replace name of new column
tgp_clinvar_updated_symbol[, Symbol := NULL]
gnomad_clinvar_updated_symbol[, Symbol := NULL]
names(tgp_clinvar_updated_symbol)[names(tgp_clinvar_updated_symbol) == "Symbol_001"] <- "Symbol"
names(gnomad_clinvar_updated_symbol)[names(gnomad_clinvar_updated_symbol) == "Symbol_001"] <- "Symbol"
# Save generated data
fwrite(tgp_clinvar, "/data/POL_data/tgp_clinvar_updated_symbol.tsv", sep="\t")
fwrite(gnomad_clinvar, "/data/POL_data/gnomad_clinvar_updated_symbol.tsv", sep="\t")

## Start from here (load necessary data)

In [ ]:
# Load previously generated data (if you have performed all the previous steps before and do not want to repeat them again)
tgp_vep_all <- fread("/data/POL_data/tgp_vep_all_subset_2021_07_16.tsv")
gnomad_vep_all <- fread("/data/POL_data/gnomad_vep_all_2021_07_16.tsv")
tgp_clinvar <- fread("/data/POL_data/tgp_clinvar_updated_symbol.tsv", sep="\t")
gnomad_clinvar <- fread("/data/POL_data/gnomad_clinvar_updated_symbol.tsv", sep="\t")
allPanelsDT3 <- fread("/data/POL_data/allPanelsDT3_2021_07_16.tsv")
omim_recessive <- fread("/data/POL_data/recessive_genes.bed", sep="\t", col.names=c("chr", "start", "end", "symbol"))
omim_recessive <- unique(omim_recessive, by = "symbol")

In [ ]:
# Check data
dim(tgp_vep_all)
dim(gnomad_vep_all)
dim(tgp_clinvar)
dim(gnomad_clinvar)
dim(allPanelsDT3)
dim(omim_recessive)

## Calculate cumulative frequencies for AR genes

In [ ]:
# Function to calculate cumulative freqecies
calculateCumFreq <- function(tgp_clinvar, gnomad_clinvar){
    tgp_clinvar[, c("POL_SNP_cum_AF", "POL_number_unique_variants") := list(sum(as.numeric(AF)), .N), by = Symbol]
    tgp_clinvar_unique <- tgp_clinvar[-which(duplicated(tgp_clinvar[,"Symbol",with = FALSE])),]
    tgp_clinvar_unique <- tgp_clinvar_unique[order(POL_SNP_cum_AF, decreasing = TRUE),
                                             c("Symbol", "POL_SNP_cum_AF",  "POL_number_unique_variants"),with=FALSE]
    gnomad_clinvar[which(gnomad_clinvar$gnomAD_AF_NFE != 0),
                   c("gnomAD_NFE_SNP_cum_AF","gnomAD_NFE_number_unique_variants") := list(sum(as.numeric(gnomAD_AF_NFE)), .N),
                   by = Symbol]
    gnomad_clinvar_unique <- gnomad_clinvar[-which(duplicated(gnomad_clinvar[,"Symbol", with = FALSE])),]
    gnomad_clinvar_unique <- gnomad_clinvar_unique[order(gnomAD_NFE_SNP_cum_AF, decreasing = TRUE),
                                                   c("Symbol", "gnomAD_NFE_SNP_cum_AF", "gnomAD_NFE_number_unique_variants"),
                                                   with = FALSE]
    list(tgp_clinvar_unique, gnomad_clinvar_unique)
}

rr <- calculateCumFreq(tgp_clinvar, gnomad_clinvar)
tgp_clinvar_unique <- rr[[1]]
gnomad_clinvar_unique <- rr[[2]]

# Merge and clean data
merge_clinvar_unique <- merge(tgp_clinvar_unique, gnomad_clinvar_unique, by = "Symbol", all = TRUE)
merge_clinvar_unique[is.na(merge_clinvar_unique)] <- 0
# Calculate cumulative allele count and number
merge_clinvar_unique$POL_SNP_cum_AC <- round(merge_clinvar_unique$POL_SNP_cum_AF*max(as.numeric(tgp_vep_all$AN)))
merge_clinvar_unique$POL_SNP_cum_AN <- max(as.numeric(tgp_vep_all$AN))
nfe_cnt <- 2*32299
merge_clinvar_unique$gnomAD_NFE_SNP_cum_AN <- nfe_cnt
merge_clinvar_unique$gnomAD_NFE_SNP_cum_AC <- round(merge_clinvar_unique$gnomAD_NFE_SNP_cum_AF*merge_clinvar_unique$gnomAD_NFE_SNP_cum_AN)
# Calculate AF fold change
merge_clinvar_unique$cum_AF_fold_change <- log2(merge_clinvar_unique$POL_SNP_cum_AF/merge_clinvar_unique$gnomAD_NFE_SNP_cum_AF)
merge_clinvar_unique[is.na(merge_clinvar_unique)] <- 0
# Add coverage information
depth_of_coverage <- fread("/data/POL_data/depth_of_coverage_gene_aggregates.tsv", sep="\t")
merge_clinvar_unique_cov <- merge(merge_clinvar_unique, depth_of_coverage, by = "Symbol", all.x = TRUE)
# Depth cutoff - keep genes that have at least 90% of the transcript covered by at least 20X, or no coverage data available
merge_clinvar_unique_cov[, FILTER := ifelse((merge_clinvar_unique_cov$"min(pct20x)" < 90 | 
                                             is.na(merge_clinvar_unique_cov$"min(pct20x)")), "LOW_DP", "PASS")]
merge_clinvar_unique <- merge_clinvar_unique_cov[FILTER == "PASS"]
# Calculate P values with Fisher test
suppressWarnings(pvals <- sapply(1:nrow(merge_clinvar_unique), function(i){
    row <- merge_clinvar_unique[i,]
    if(!is.finite(row$gnomAD_NFE_SNP_cum_AC)){return (NA)}
    M <- as.table(rbind(c(row$POL_SNP_cum_AC, (row$POL_SNP_cum_AN- row$POL_SNP_cum_AC)),
                        c(row$gnomAD_NFE_SNP_cum_AC, (row$gnomAD_NFE_SNP_cum_AN - row$gnomAD_NFE_SNP_cum_AC))))
    dimnames(M) <- list(pop = c("PL", "NFE"),  value = c("AC", "AN"))
    (fisher <- fisher.test(M))
    fisher$p.value
    }))
merge_clinvar_unique$fisher_p <- pvals
# Bonferonni correction
merge_clinvar_unique$fisher_p_corr <- merge_clinvar_unique$fisher_p*nrow(merge_clinvar_unique)
# Apply Benjamini-Hochberg p-value correction for multiple hypothesis testing
merge_clinvar_unique$fisher_q <- p.adjust(merge_clinvar_unique$fisher_p, "fdr")
# Save generated data
setcolorder(merge_clinvar_unique, c('Symbol','POL_number_unique_variants','POL_SNP_cum_AF','POL_SNP_cum_AC','POL_SNP_cum_AN',
                                    'gnomAD_NFE_number_unique_variants','gnomAD_NFE_SNP_cum_AF','gnomAD_NFE_SNP_cum_AN',
                                    'gnomAD_NFE_SNP_cum_AC','cum_AF_fold_change','fisher_p','fisher_p_corr', 'fisher_q', 'FILTER'))
# Add genes discarded on basis of FILTER
merge_clinvar_unique <- rbind(merge_clinvar_unique, merge_clinvar_unique_cov[FILTER == "LOW_DP"], fill=TRUE)
fwrite(merge_clinvar_unique, "/data/POL_data/Supplementary_Table_S5_part1.tsv", sep="\t")

## Cumulative frequency differences visualisation

In [ ]:
tcu <- merge_clinvar_unique[which(merge_clinvar_unique$fisher_q < 0.05)]
nrow(tcu)
head(tcu)

In [ ]:
set.seed(42)
p <- ggplot(tcu, aes(POL_SNP_cum_AF, gnomAD_NFE_SNP_cum_AF)) +
    geom_point() +
    geom_abline(intercept = 0, slope = 1) +
    ylab("Per-gene cumulative allele frequencies in gnomAD NFE population") +
    xlab("Per-gene cumulative allele frequencies in Polish population")
p <- p +  geom_label_repel(
    aes(fill = "red", label = Symbol),
    fontface = 'bold.italic',
    color = 'white',
    size = 4,
    segment.color = "black",
    max.overlaps = 10,
    min.segment.length = 0,
    box.padding = unit(0.1, "lines"),
    point.padding = unit(0.1, "lines")
)
tiff(filename="/data/POL_data/Figure_8.tiff", width=520, height=520)
p + theme(legend.position = "none") + theme(plot.title = element_text(size=11))
dev.off()

In [ ]:
# Select data
allPanelsDT3$POL_SNP_cum_AF <- merge_clinvar_unique$POL_SNP_cum_AF[match(allPanelsDT3$"Gene Symbol",
                                                                         merge_clinvar_unique$Symbol)]
allPanelsDT3$POL_number_unique_variants <- merge_clinvar_unique$POL_SNP_cum_AC[match(allPanelsDT3$"Gene Symbol",
                                                                                     merge_clinvar_unique$Symbol)]

allPanelsDT3$gnomAD_NFE_SNP_cum_AF <- merge_clinvar_unique$gnomAD_NFE_SNP_cum_AF[match(allPanelsDT3$"Gene Symbol",
                                                                                       merge_clinvar_unique$Symbol)]
allPanelsDT3$gnomAD_NFE_number_unique_variants <- merge_clinvar_unique$gnomAD_NFE_SNP_cum_AC[match(allPanelsDT3$"Gene Symbol",
                                                                                                   merge_clinvar_unique$Symbol)]

# Calculate cumulative AF/AC/AN 
allPanelsDT3[,POL_SNP_cum_AF_sum := sum(POL_SNP_cum_AF, na.rm=TRUE), by=Level4]
allPanelsDT3[,POL_SNP_cum_AC_sum := round(allPanelsDT3$POL_SNP_cum_AF_sum*max(as.numeric(tgp_vep_all$AN)))]
allPanelsDT3[,POL_SNP_cum_AN_sum := max(as.numeric(tgp_vep_all$AN))]
allPanelsDT3[,POL_number_unique_variants_sum := sum(POL_number_unique_variants, na.rm=TRUE), by=Level4]

nfe_cnt <- 2*32299
allPanelsDT3[,gnomAD_NFE_SNP_cum_AF_sum := sum(gnomAD_NFE_SNP_cum_AF, na.rm=TRUE), by=Level4]
allPanelsDT3[,gnomAD_NFE_SNP_cum_AC_sum := round(allPanelsDT3$gnomAD_NFE_SNP_cum_AF_sum*nfe_cnt)]
allPanelsDT3[,gnomAD_NFE_SNP_cum_AN_sum := nfe_cnt]
allPanelsDT3[,gnomAD_NFE_number_unique_variants_sum := sum(gnomAD_NFE_number_unique_variants, na.rm=TRUE), by=Level4]
# Calculate number of AR genes in given panel
allPanelsDT3[,genes_in_panel := .N, by=Level4]

# Select unique panels
allPanelsDT3_unique <- allPanelsDT3[-which(duplicated(Level4)),]
allPanelsDT3_unique$Symbol <- allPanelsDT3_unique$"Gene Symbol"
allPanelsDT3_unique$cum_AF_fold_change <- log2(allPanelsDT3_unique$POL_SNP_cum_AF_sum /
                                               allPanelsDT3_unique$gnomAD_NFE_SNP_cum_AF_sum)

# Clean +/- Inf in fold_change calculations
allPanelsDT3_unique[which(!is.finite(cum_AF_fold_change))]$cum_AF_fold_change <- 0
# Taking into considerarion of number of genes in given panel 
allPanelsDT3_unique[, POL_SNP_cum_AN_sum_adjusted := genes_in_panel * POL_SNP_cum_AN_sum]
allPanelsDT3_unique[, gnomAD_NFE_SNP_cum_AN_sum_adjusted := genes_in_panel * gnomAD_NFE_SNP_cum_AN_sum]
# Calculate Fisher P-value
suppressWarnings(pvals <- sapply(1:nrow(allPanelsDT3_unique), function(i){
    row <- allPanelsDT3_unique[i,]
    if(row$POL_SNP_cum_AC_sum == 0){return (NA)}
    # Taking into considerarion of number of genes in given panel 
    M <- as.table(rbind(c(row$POL_SNP_cum_AC_sum, (row$POL_SNP_cum_AN_sum_adjusted - row$POL_SNP_cum_AC_sum)),
                        c(row$gnomAD_NFE_SNP_cum_AC_sum, (row$gnomAD_NFE_SNP_cum_AN_sum_adjusted - row$gnomAD_NFE_SNP_cum_AC_sum))))
    dimnames(M) <- list(pop = c("PL", "NFE"),  value = c("AC", "AN"))
    fisher <- fisher.test(unlist(M))
    fisher$p.value
}))
allPanelsDT3_unique[, POL_SNP_cum_AN_sum_adjusted := genes_in_panel * POL_SNP_cum_AN_sum]
allPanelsDT3_unique[, gnomAD_NFE_SNP_cum_AN_sum_adjusted := genes_in_panel * gnomAD_NFE_SNP_cum_AN_sum]
allPanelsDT3_unique$fisher_p <- pvals
allPanelsDT3_unique$fisher_p_corr <- allPanelsDT3_unique$fisher_p*nrow(allPanelsDT3_unique)
# Apply Benjamini-Hochberg p-value correction for multiple hypothesis testing
allPanelsDT3_unique$fisher_q <- p.adjust(allPanelsDT3_unique$fisher_p, "fdr")
# Save generated data
fwrite(allPanelsDT3_unique, "/data/POL_data/allPanelsDT3_unique_2021_07_16.tsv", sep="\t")

In [ ]:
head(allPanelsDT3_unique)

In [ ]:
# Generate Supplementary Table of gene panels
selected_columns <- c("Level4", "POL_SNP_cum_AF_sum", "gnomAD_NFE_SNP_cum_AF_sum", "cum_AF_fold_change", "POL_SNP_cum_AC_sum",
                      "POL_SNP_cum_AN_sum_adjusted", "gnomAD_NFE_SNP_cum_AC_sum", "gnomAD_NFE_SNP_cum_AN_sum_adjusted",
                      "POL_number_unique_variants_sum", "gnomAD_NFE_number_unique_variants_sum", "genes_in_panel",
                      "fisher_p", "fisher_p_corr", "fisher_q")
# Select relevant columns
SuppTable_recessive_panels <- allPanelsDT3_unique[, ..selected_columns]
names(SuppTable_recessive_panels)[names(SuppTable_recessive_panels) == 'Level4'] <- 'Gene_panel'
# Clean up table
setcolorder(SuppTable_recessive_panels, c('Gene_panel', "genes_in_panel", 'POL_SNP_cum_AF_sum', 'POL_SNP_cum_AC_sum', 'POL_SNP_cum_AN_sum_adjusted', 'POL_number_unique_variants_sum',
                'gnomAD_NFE_SNP_cum_AF_sum', 'gnomAD_NFE_SNP_cum_AC_sum', 'gnomAD_NFE_SNP_cum_AN_sum_adjusted',
                'gnomAD_NFE_number_unique_variants_sum', 'cum_AF_fold_change', 'fisher_p', 'fisher_p_corr', 'fisher_q'))
setorder(SuppTable_recessive_panels, fisher_p)
replaceNA = function(DT) {
  for (j in seq_len(ncol(DT)))
    set(DT,which(is.na(DT[[j]])),j,0)
}
replaceNA(SuppTable_recessive_panels)
SuppTable_recessive_panels$fisher_q <- ifelse(SuppTable_recessive_panels$cum_AF_fold_change == 0, 1, SuppTable_recessive_panels$fisher_q)
# Save generated data
fwrite(SuppTable_recessive_panels, "/data/POL_data/Supplementary_Table_S6.tsv", sep="\t")

In [470]:
# Select significantly changed panels
selPanels <- SuppTable_recessive_panels[which(fisher_q <= 0.05 & abs(cum_AF_fold_change) > 1 & POL_number_unique_variants_sum >= 3),]
setorder(selPanels, cum_AF_fold_change)
selPanels

Gene_panel,genes_in_panel,POL_SNP_cum_AF_sum,POL_SNP_cum_AC_sum,POL_SNP_cum_AN_sum_adjusted,POL_number_unique_variants_sum,gnomAD_NFE_SNP_cum_AF_sum,gnomAD_NFE_SNP_cum_AC_sum,gnomAD_NFE_SNP_cum_AN_sum_adjusted,gnomAD_NFE_number_unique_variants_sum,cum_AF_fold_change,fisher_p,fisher_p_corr,fisher_q
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mitochondrial disorder with complex I deficiency,27,0.001394052,3,58104,3,0.0072803439,470,1744146,470,-2.384722,0.00034398386486857,0.085307998487406,0.0035053593848512
Pyruvate dehydrogenase (PDH) deficiency,23,0.001394484,3,49496,3,0.0054985950,355,1485754,355,-1.979332,0.00628171001072979,1.557864082660988,0.0362878556498972
Hydrocephalus,39,0.008829862,19,83928,19,0.0230871066,1491,2519322,1491,-1.386625,0.00000177231273243,0.000439533557643,0.0000344795386128
Limb girdle muscular dystrophy,32,0.015800557,34,68864,34,0.0396644381,2562,2067136,2563,-1.327871,0.00000000071986324,0.000000178526083,0.0000000308101466
GI tract tumours,6,0.004183020,9,12912,9,0.0103032797,666,387588,665,-1.300487,0.00301837509128305,0.748557022638196,0.0215310756511524
Rare anaemia,41,0.006971124,15,88232,15,0.0168698126,1090,2648518,1089,-1.274981,0.00015731800715811,0.039014865775210,0.0019803560901079
Colorectal cancer pertinent cancer susceptibility,2,0.003718336,8,4304,8,0.0088889847,574,129196,574,-1.257362,0.00902475681189842,2.238139689350807,0.0439522662838934
Inherited polyposis,2,0.003718336,8,4304,8,0.0088889847,574,129196,574,-1.257362,0.00902475681189842,2.238139689350807,0.0439522662838934
Additional findings health related,1,0.003718336,8,2152,8,0.0085482395,552,64598,552,-1.200970,0.01126297881939304,2.793218747209474,0.0478090553929061


In [467]:
# Generate gene panels figure
set.seed(42)
selPanels[, panelId := seq(1:nrow(selPanels))]
p <- ggplot(selPanels, aes(POL_SNP_cum_AF_sum, gnomAD_NFE_SNP_cum_AF_sum, label=panelId)) +
  geom_point() +
  geom_abline() + 
  xlim(0,0.04) + ylim(0,0.04)+
  ylab("Per-panel cumulative allele frequencies in gnomAD NFE population")+
  xlab("Per-panel cumulative allele frequencies in Polish population")

p <- p + geom_label_repel(
  fontface = 'bold.italic',
  size = 6,
  segment.color = "black",
  min.segment.length = 0,
  box.padding = unit(0.1, "lines"),
  point.padding = unit(0.1, "lines"),
  max.overlaps = Inf
)
p <- p + labs(caption="
1. Mitochondrial disorder with complex I deficiency,
2. Pyruvate dehydrogenase (PDH) deficiency, 3. Hydrocephalus,
4. Limb girdle muscular dystrophy, 5. GI tract tumours, 6. Rare anaemia,
7. Colorectal cancer pertinent cancer susceptibility, 8. Inherited polyposis,
9. Additional findings health related, 10. Additional findings health related - adults,
11. Additional findings health related - adult specific,
12. Additional findings health related - CNV analysis adults,
13. Additional findings health related - CNV analysis adult specific,
14. Nephrocalcinosis or nephrolithiasis, 15. Malformations of cortical development,
16. Fetal hydrops, 17. Cerebellar hypoplasia,
18. Palmoplantar keratoderma and erythrokeratodermas,
19. Epidermolysis bullosa and congenital skin fragility, 20. Epidermolysis bullosa,
21. Thoracic aortic aneurysm and dissection, 22. Peeling skin syndrome,
23. Optic neuropathy, 24. Autosomal recessive primary hypertrophic osteoarthropathy")
tiff(filename="/data/POL_data/Figure_9.tiff", width=600, height=800)
p + theme(plot.caption = element_text(size = 13, hjust = 0))
dev.off()

png 
  2

# Structural Variants
## POL SV data

In [ ]:
tgp_sv_af <- fread("/data/POL_data/sv_multisample_210716.smoove.square.anno.unrelated.nogt.vcf.gz", skip="#CHROM")
# Split INFO field from VCF into separate columns
info <- strsplit(tgp_sv_af$INFO, ";")
tgp_sv_af$POS <- as.numeric(tgp_sv_af$POS)
tgp_sv_af$SVTYPE <- (sapply(info, function(x){ gsub("SVTYPE=" , "",x[grep("SVTYPE=",x)])}))
tgp_sv_af$END <- (sapply(info, function(x){ gsub("^END=" , "",x[grep("^END=",x)])}))
tgp_sv_af$AF <- (sapply(info, function(x){ gsub("AF=" , "",x[grep("AF=",x)])}))
tgp_sv_af$SVLEN <- (sapply(info, function(x){ gsub("SVLEN=" , "",x[grep("SVLEN=",x)])}))
tgp_sv_af$END <- as.numeric(tgp_sv_af$END)
tgp_sv_af$LEN <- tgp_sv_af$END - tgp_sv_af$POS
# Select deletions, duplications and inversions with PASS only
tgp_sv_af <- tgp_sv_af[which(tgp_sv_af$SVTYPE %in% c("DEL", "DUP", "INV")), ]
tgp_sv_af <- tgp_sv_af[which(tgp_sv_af$FILTER == "PASS"), ]
tgp_sv_af_gr <- GRanges(tgp_sv_af$`#CHROM`, IRanges(tgp_sv_af$POS, tgp_sv_af$END))

## gnomAD v2.1 SV NFE data

In [ ]:
gnomad_sv_af <- fread("/data/POL_data/gnomad_v2.1_sv.sites_chr.vcf.gz", skip="#CHROM")
# Split INFO field from VCF into separate columns
info <- strsplit(gnomad_sv_af$INFO, ";")
gnomad_sv_af$POS <- as.numeric(gnomad_sv_af$POS)
gnomad_sv_af$SVTYPE <- (sapply(info, function(x){gsub("SVTYPE=" , "",x[grep("SVTYPE=",x)])}))
gnomad_sv_af$END <- (sapply(info, function(x){gsub("^END=" , "",x[grep("^END=",x)])}))
gnomad_sv_af$AF <- (sapply(info, function(x){gsub("EUR_AF=" , "",x[grep("EUR_AF=",x)])}))
gnomad_sv_af$AF <- as.numeric(lapply(gnomad_sv_af$AF, `[[`, 1))
gnomad_sv_af$SVLEN <- (sapply(info, function(x){gsub("SVLEN=" , "",x[grep("SVLEN=",x)])}))
gnomad_sv_af$END <- as.numeric(gnomad_sv_af$END)
gnomad_sv_af$LEN <- gnomad_sv_af$END - gnomad_sv_af$POS
# Select deletions, duplications and inversions with PASS only
gnomad_sv_af <- gnomad_sv_af[which(gnomad_sv_af$SVTYPE %in% c("DEL", "DUP", "INV")), ]
gnomad_sv_af <- gnomad_sv_af[which(gnomad_sv_af$FILTER == "PASS"), ]
gnomad_sv_af_gr <- GRanges(gnomad_sv_af$`#CHROM`, IRanges(gnomad_sv_af$POS, gnomad_sv_af$END))

## Select recessive disease genes from refseq and OMIM

In [ ]:
refSeq <- fread("/data/POL_data/hg38.ncbiRefSeq.no.pred.gtf")
setnames(refSeq, c("chr", "source", "level", "start", "end", "V6", "V7", "V8", "INFO"))
info2 <-  strsplit(refSeq$INFO, ";")
refSeq$gene_name <- (sapply(info2, function(x){x[grep("gene_name",x)]}))
refSeq$gene_name <- gsub ("\"", "",gsub(" gene_name " , "",refSeq$gene_name))
omim_recessive <- fread("/data/POL_data/recessive_genes.bed", sep="\t", col.names=c("chr", "start", "end", "symbol"))
omim_recessive <- unique(omim_recessive, by = "symbol")
refSeq_recessive <- refSeq[which(refSeq$gene_name %in% omim_recessive$symbol ),]
refSeq_recessive <- refSeq_recessive[ which(refSeq_recessive$chr %in% paste0("chr",1:22)),]
refSeq_recessive[,unique_chr:=length(unique(chr)),by=gene_name]
length(unique(refSeq_recessive$gene_name))

## Select Loss of Function SVs for recessive disease genes
### POL cohort

In [ ]:
TGP_SV_recessive <- rbindlist(mclapply(unique(refSeq_recessive$gene_name), function(x){
    refSeq_recessive[which(refSeq_recessive$gene_name == x  ),]
    selRefSeq_exons <- refSeq_recessive[which(refSeq_recessive$gene_name == x  & refSeq_recessive$level == "CDS"),]
    if (nrow(selRefSeq_exons)==0){return (NULL)}
    selRefSeq_exons_gr <- GRanges(selRefSeq_exons$chr, IRanges(selRefSeq_exons$start, selRefSeq_exons$end))
    selRefSeq_gene_gr <- GRanges(selRefSeq_exons$chr[1], IRanges(min(selRefSeq_exons$start), max(selRefSeq_exons$end)))
    mm_gene <- findOverlaps(tgp_sv_af_gr, selRefSeq_gene_gr)
    if (length(mm_gene) ==0) {return(NULL)}
    svs_idx <- as.matrix(mm_gene)[,1]
    # iterate over all SVs that intersect with a coding part of a gene
    rbindlist(lapply(svs_idx, function(i){
        sel_sv <- tgp_sv_af[i,]
        sel_tgp_sv_af_gr <- tgp_sv_af_gr[i,]
        mm_exons <- findOverlaps(sel_tgp_sv_af_gr,selRefSeq_gene_gr )
        any_overlap_with_exons <- length(mm_exons) > 0
        both_bp_within_gene <- sel_sv$POS > min(selRefSeq_exons$start) & sel_sv$POS < max(selRefSeq_exons$end) & 
                               sel_sv$END > min(selRefSeq_exons$start) & sel_sv$END < max(selRefSeq_exons$end)

        exactly_one_breakpoint_within_gene <- ((sel_sv$POS > min(selRefSeq_exons$start) &  sel_sv$POS < max(selRefSeq_exons$end)) & 
                                              !(sel_sv$END > min(selRefSeq_exons$start) &  sel_sv$END < max(selRefSeq_exons$end))) |
                                              (!(sel_sv$POS > min(selRefSeq_exons$start) &  sel_sv$POS < max(selRefSeq_exons$end)) & 
                                              (sel_sv$END > min(selRefSeq_exons$start) &  sel_sv$END < max(selRefSeq_exons$end)))
# criteria from https://gnomad.broadinstitute.org/news/images/2019/03/structural-variants-in-gnomad/gene_annotation_schematics.png
        lof_filter <- (sel_sv$SVTYPE == "DEL" & any_overlap_with_exons) | 
                      (sel_sv$SVTYPE == "DUP" & any_overlap_with_exons & both_bp_within_gene ) | 
                      (sel_sv$SVTYPE == "INV" & any_overlap_with_exons & both_bp_within_gene ) | 
                      (sel_sv$SVTYPE == "INV" & exactly_one_breakpoint_within_gene) 

        if(lof_filter){
            return(data.table(SYMBOL=x, SVTYPE=sel_sv$SVTYPE, AF=sel_sv$AF, GENELEN= max(selRefSeq_exons$end) - min(selRefSeq_exons$start)) )
        }
        return (NULL)
    }))
},mc.cores=40))

### gnomAD cohort

In [ ]:
gnomAD_SV_recessive <- rbindlist(mclapply(unique(refSeq_recessive$gene_name), function(x){
    refSeq_recessive[which(refSeq_recessive$gene_name == x  ),]
    selRefSeq_exons <- refSeq_recessive[which(refSeq_recessive$gene_name == x  & refSeq_recessive$level == "CDS"),]
    if (nrow(selRefSeq_exons)==0){return (NULL)}
    selRefSeq_exons_gr <- GRanges(selRefSeq_exons$chr, IRanges(selRefSeq_exons$start, selRefSeq_exons$end))
    selRefSeq_gene_gr <- GRanges(selRefSeq_exons$chr[1], IRanges(min(selRefSeq_exons$start), max(selRefSeq_exons$end)))
    mm_gene <- findOverlaps(gnomad_sv_af_gr, selRefSeq_gene_gr)
    if (length(mm_gene) ==0) {return(NULL)}
    svs_idx <- as.matrix(mm_gene)[,1]
    # iterate over all SVs that intersect with a coding part of a gene
    rbindlist(lapply(svs_idx, function(i){
        sel_sv <- gnomad_sv_af[i,]
        sel_gnomad_sv_af_gr <- gnomad_sv_af_gr[i,]
        mm_exons <- findOverlaps(sel_gnomad_sv_af_gr,selRefSeq_gene_gr )
        any_overlap_with_exons <- length(mm_exons) > 0
        both_bp_within_gene <- sel_sv$POS > min(selRefSeq_exons$start) &  sel_sv$POS < max(selRefSeq_exons$end) & 
                                sel_sv$END > min(selRefSeq_exons$start) &  sel_sv$END < max(selRefSeq_exons$end)

        exactly_one_breakpoint_within_gene <- ((sel_sv$POS > min(selRefSeq_exons$start) &  sel_sv$POS < max(selRefSeq_exons$end)) & 
                                              !(sel_sv$END > min(selRefSeq_exons$start) &  sel_sv$END < max(selRefSeq_exons$end))) |
                                              (!(sel_sv$POS > min(selRefSeq_exons$start) &  sel_sv$POS < max(selRefSeq_exons$end)) & 
                                              (sel_sv$END > min(selRefSeq_exons$start) &  sel_sv$END < max(selRefSeq_exons$end)))
# criteria from https://gnomad.broadinstitute.org/news/images/2019/03/structural-variants-in-gnomad/gene_annotation_schematics.png
        lof_filter <- (sel_sv$SVTYPE == "DEL" & any_overlap_with_exons) | 
                      (sel_sv$SVTYPE == "DUP" & any_overlap_with_exons & both_bp_within_gene ) | 
                      (sel_sv$SVTYPE == "INV" & any_overlap_with_exons & both_bp_within_gene ) | 
                      (sel_sv$SVTYPE == "INV" & exactly_one_breakpoint_within_gene) 

        if(lof_filter){
            return (data.table(SYMBOL= x, SVTYPE=sel_sv$SVTYPE, AF=sel_sv$AF , GENELEN=max(selRefSeq_exons$end) - min(selRefSeq_exons$start)) )
        }
        return (NULL)
    }))
},mc.cores=40))

In [ ]:
# Change to wide format
TGP_SV_recessive_wide <- dcast(TGP_SV_recessive2, SYMBOL ~ SVTYPE, value.var = "cum_af")
gnomAD_SV_recessive_wide <- dcast(gnomAD_SV_recessive2, SYMBOL ~ SVTYPE, value.var = "cum_af")
TGP_SV_recessive_wide[is.na(TGP_SV_recessive_wide)] <- 0
gnomAD_SV_recessive_wide[is.na(gnomAD_SV_recessive_wide)] <- 0
# Remove gene length and duplicated total cum. columns
cum_cols <- c("DEL", "DUP", "INV")
TGP_SV_recessive_wide[, total_cum_SV_af := rowSums(.SD), .SDcols = cum_cols]
gnomAD_SV_recessive_wide[, total_cum_SV_af := rowSums(.SD), .SDcols = cum_cols]
# Change column names
colnames_old <- c('SYMBOL','DEL','DUP','INV','total_cum_SV_af')
colnames_new_TGP <- c('Symbol','POL_SV_del_cum_AF','POL_SV_dup_cum_AF','POL_SV_inv_cum_AF','POL_SV_total_cum_AF')
colnames_new_gnomAD <- c('Symbol','gnomAD_NFE_SV_del_cum_AF','gnomAD_NFE_SV_dup_cum_AF','gnomAD_NFE_SV_inv_cum_AF',
                         'gnomAD_NFE_SV_total_cum_AF')
setnames(TGP_SV_recessive_wide, colnames_old, colnames_new_TGP)
setnames(gnomAD_SV_recessive_wide, colnames_old, colnames_new_gnomAD)
fwrite(TGP_SV_recessive_wide, "/data/POL_data/TGP_SV_recessive_wide.tsv", sep="\t")
fwrite(gnomAD_SV_recessive_wide, "/data/POL_data/gnomAD_SV_recessive_wide.tsv", sep="\t")

# Combine SNV and SV variants data

In [ ]:
TGP_SV_recessive <- fread("/data/POL_data/TGP_SV_recessive_wide.tsv")
gnomAD_SV_recessive <- fread("/data/POL_data/gnomAD_SV_recessive_wide.tsv")
# Merge SNV and SV data
combined_recessive_variants <- merge(merge_clinvar_unique, gnomAD_SV_recessive, by = "Symbol", all = TRUE)
combined_recessive_variants2 <- merge(combined_recessive_variants, TGP_SV_recessive, by = "Symbol", all = TRUE)
# Clean +/- Inf in fold_change calculations
combined_recessive_variants2[which(!is.finite(cum_AF_fold_change))]$cum_AF_fold_change <- 0
# replace NA with 0
replaceNA(combined_recessive_variants2)
# Replace fisher_q value for genes with 0 SNP variants in both POL and gnomAD cohort
combined_recessive_variants2$fisher_q <- ifelse((combined_recessive_variants2$POL_number_unique_variants == 0 &
                                                combined_recessive_variants2$gnomAD_NFE_number_unique_variants == 0),
                                                 1, combined_recessive_variants2$fisher_q)
combined_recessive_variants2$fisher_q <- ifelse(combined_recessive_variants2$FILTER == "LOW_DP", as.character("NA"), combined_recessive_variants2$fisher_q)
# Remove coverage information and add it again to calculate for all genes
combined_recessive_variants2[, c("FILTER", "min(mean_depth)", "avg(mean_depth)", "max(mean_depth)", "min(pct10x)",
                                 "avg(pct10x)", "max(pct10x)", "min(pct20x)", "avg(pct20x)", "max(pct20x)") := NULL]
combined_recessive_variants2 <- merge(combined_recessive_variants2, depth_of_coverage, by = "Symbol", all.x = TRUE)
# Depth cutoff - keep genes that have at least 90% of the transcript covered by at least 20X, or no coverage data available
combined_recessive_variants2[, FILTER := ifelse((combined_recessive_variants2$"min(pct20x)" < 90 | is.na(combined_recessive_variants2$"min(pct20x)")), "LOW_DP", "PASS")]

In [ ]:
fwrite(combined_recessive_variants2, "/data/POL_data/Supplementary_Table_S5.tsv", sep="\t", na=0)